In [1]:
import rdkit 
from rdkit.Chem import MolFromSmiles as get_mol
from rdkit.Chem import rdDistGeom
import numpy as np 
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
from torch.utils.data import Dataset, DataLoader 
%run utils.py


In [52]:
smi_list = read_smi_txt('data/ADAGRASIB_SMILES.txt')


In [7]:
import rdkit 

def get_atom_pos(smi) :
    mol = rdkit.Chem.MolFromSmiles(smi) 
    mol_h = rdkit.Chem.AddHs(mol) 
    rdkit.Chem.rdDistGeom.EmbedMolecule(mol_h) 
    conformer = mol_h.GetConformer()
    return conformer.GetPositions()

get_atom_pos('Cn1ncc(c1)B1OC(C(O1)(C)C)(C)C')

array([[ 5.13428525e+00,  8.03563408e-02, -4.22529056e-01],
       [ 3.91281401e+00,  1.02843500e-01,  3.79922460e-01],
       [ 3.82253888e+00,  2.92577087e-03,  1.69567731e+00],
       [ 2.55248614e+00,  6.06241171e-02,  2.10546696e+00],
       [ 1.78698577e+00,  2.07780096e-01,  9.62442942e-01],
       [ 2.66372204e+00,  2.31159807e-01, -1.02329414e-01],
       [ 2.46716282e-01,  3.25711245e-01,  8.66145325e-01],
       [-7.87452754e-01, -4.34960922e-01,  1.56452935e+00],
       [-1.85766799e+00, -4.27650432e-01,  6.54498402e-01],
       [-1.54121787e+00,  5.75471599e-01, -3.98223975e-01],
       [-4.78464886e-01,  1.34352171e+00,  1.11401100e-01],
       [-2.66929582e+00,  1.50378776e+00, -7.26213151e-01],
       [-9.87135504e-01, -7.02793953e-02, -1.67530145e+00],
       [-3.09647657e+00, -9.87956907e-02,  1.45655048e+00],
       [-1.98722607e+00, -1.83254455e+00,  1.28396703e-01],
       [ 5.40938382e+00,  1.06659971e+00, -8.06717681e-01],
       [ 5.96245888e+00, -2.65464276e-01

In [87]:
get_atom_pos('CCC')

array([[ 1.21899678, -0.24220362,  0.14122404],
       [ 0.03067563,  0.68832595, -0.0942759 ],
       [-1.19133694, -0.22516084, -0.14922066],
       [ 1.06315635, -1.13815056, -0.50579798],
       [ 2.17813328,  0.24361292, -0.00256337],
       [ 1.05595285, -0.61866047,  1.19153562],
       [ 0.14271289,  1.30405858, -0.98547916],
       [-0.01438078,  1.37308502,  0.78767599],
       [-1.92642986,  0.10232931, -0.89381998],
       [-0.91176429, -1.27547443, -0.35848626],
       [-1.64571591, -0.21176186,  0.86920767]])

In [74]:
mol = get_mol('CCC')
mol_h = rdkit.Chem.AddHs(mol) 
rdDistGeom.EmbedMolecule(mol_h)
conf = mol_h.GetConformer()
conf.GetPositions()

array([[ 1.21899678, -0.24220362,  0.14122404],
       [ 0.03067563,  0.68832595, -0.0942759 ],
       [-1.19133694, -0.22516084, -0.14922066],
       [ 1.06315635, -1.13815056, -0.50579798],
       [ 2.17813328,  0.24361292, -0.00256337],
       [ 1.05595285, -0.61866047,  1.19153562],
       [ 0.14271289,  1.30405858, -0.98547916],
       [-0.01438078,  1.37308502,  0.78767599],
       [-1.92642986,  0.10232931, -0.89381998],
       [-0.91176429, -1.27547443, -0.35848626],
       [-1.64571591, -0.21176186,  0.86920767]])

In [57]:
smi_list[0]

'Cn1ncc(c1)B1OC(C(O1)(C)C)(C)C\n'

In [45]:
test('FC(COS(=O)(=O)c1ccc(cc1)C)(F)F')

ValueError: Bad Conformer Id

In [40]:
smi_list

['Cn1ncc(c1)B1OC(C(O1)(C)C)(C)C\n',
 'FC(COS(=O)(=O)c1ccc(cc1)C)(F)F\n',
 'O=Cc1ccc(o1)c1ccc(cc1)S(=O)(=O)N\n',
 'N#Cc1ccc(cc1)NC(=O)/C=C/C(=O)O\n',
 'CCOC(=O)/C(=C/c1ccco1)/C#N\n',
 'COc1ccc(cc1)/C=C/C(=O)N1CCOCC1\n',
 'COc1ccc(cc1)/C=C/C(=O)N1CCCCC1\n',
 'O=C(/C=C/C(=O)N)NCc1ccccc1\n',
 'COc1ccccc1NC(=O)/C=C/C(=O)N\n',
 'O=C(Nc1ccccc1Cl)/C=C/C(=O)N\n',
 'CCOc1ccccc1NC(=O)/C=C/C(=O)N\n',
 'O=C(Nc1ccc(cc1)C)/C=C/C(=O)N\n',
 'CN(c1ccc(cc1)NC(=O)/C=C/C(=O)O)C\n',
 'S=C1NC(=O)/C(=C\\c2cccnc2)/N1\n',
 'COC(=O)C(Sc1nc2c([nH]1)cccc2)C\n',
 'C=CCNc1[nH]c2c(n1)n(C)c(=O)[nH]c2=O\n',
 'N#Cc1cccc(c1C#N)[N+](=O)[O-]\n',
 'Cc1nc(N/N=C/c2ccc(c(c2)O)O)nc(c1)O\n',
 '[O-]S(=O)(=O)c1nc2c(s1)cccc2.[K+]\n',
 'N#CC(=C(Cl)Cl)NC(=O)C(C)(C)C\n',
 'O=C(Nc1ccc(cc1)C)/C=C/C(=O)O\n',
 'O=C(NC1CCCCC1)/C=C/C(=O)O\n',
 'Oc1ccc(cc1)/C=[N+](/c1ccccc1)\\[O-]\n',
 'O=C(Nc1ccc(cc1)N1CCOCC1)/C=C/C(=O)O\n',
 'OC(=O)C12CC3CC(C1)CC(C2)(C3)Br\n',
 'CCOC(=O)CSc1nnc2c(n1)[nH]c1c2cccc1\n',
 'COc1cc(OC)ccc1C=C1C(=O)NC(=S)NC1=O\n'

In [36]:
test = get_mol('Cn1ncc(c1)B1OC(C(O1)(C)C)(C)C')
test.GetConformer()

ValueError: Bad Conformer Id

In [32]:
test('Cn1ncc(c1)B1OC(C(O1)(C)C)(C)C')

ValueError: Bad Conformer Id

In [26]:
a = rdkit.Chem.MolFromSmiles('Cn1ncc(c1)B1OC(C(O1)(C)C)(C)C')
print(a)

In [24]:
coor_list = get_coor('data/ADAGRASIB_COOR.sdf')

KeyboardInterrupt: 

In [14]:
smi_list[0]

'Cn1ncc(c1)B1OC(C(O1)(C)C)(C)C'

In [15]:
coor_list[0]

tensor([[ 4.8285, -1.0040,  0.2024],
        [ 3.5776, -0.2572,  0.0479],
        [ 3.4435,  1.1346,  0.1047],
        [ 2.1893,  1.4450, -0.0747],
        [ 1.4645,  0.2475, -0.2554],
        [ 2.3676, -0.7919, -0.1777],
        [-0.0805,  0.1225, -0.5047],
        [-1.0404,  1.1849, -0.5963],
        [-2.2048,  0.6858,  0.0949],
        [-2.0701, -0.8493,  0.0305],
        [-0.8409, -1.0789, -0.6970],
        [-1.9777, -1.4359,  1.4405],
        [-3.2539, -1.4571, -0.7246],
        [-2.2055,  1.1603,  1.5494],
        [-3.4816,  1.1392, -0.6157]])